# REMIND Electricity Sector Biosphere Flows

Calculate all biosphere flow coefficients for the REMIND electricity sector. **Values are given per kWh**.

In [144]:
from progressbar import progressbar
import helpers.eimod as eimod
import helpers.ei2rmnd as ei2rmnd

In [3]:
%run initialize_notebook.ipynb

In [4]:
databases

Databases dictionary with 9 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

## Bioflows without double counting

Calculate impact for an electricity generating technology in the REMIND power sector excluding *other* activities that are part of the electricity sector to avoid double counting.

In [140]:
reload(ei2rmnd)

<module 'helpers.ei2rmnd' from '/home/alois/git/rmnd-lca/notebooks/helpers/ei2rmnd.py'>

In [153]:
def rmnd_bioflows_without_double_counting(region, scenario="BAU", year=2030, tech_primers={}):
    """ Extract bioflows for the REMIND electricity sector in a REMIND region.
    
    In this version, for every REMIND technology, we exclude all other technologies
    when performing the LCA to avoid double counting.
        
    A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
    REMIND technology x.
    """
    eidb_name = ei2rmnd.get_REMIND_database_name(scenario, year)
    eidb = Database(eidb_name)

    result = {}
    bio_names = []
    regions = ei2rmnd.ei_locations_in_remind_region(region)
    print("Found following regions for {}: {}".format(region, regions))
    
    actvts_by_tech = {}
    
    # populate activity dictionary
    print("Collecting ecoinvent activities:")
    for tech, act_list in progressbar(
        eimod.available_electricity_generating_technologies.items(), 
        prefix="Collecting ecoinvent activities: "):
        
        actvts_by_tech[tech] = [act 
                                for act_name in act_list 
                                for act in ei2rmnd.find_activities_by_name(act_name, eidb)]
    
    # flat activity list (needed for excludes)
    all_actvts = [
        act 
        for act_lst in actvts_by_tech.values()
        for act in act_lst] 

    for tech, all_tech_actvts in actvts_by_tech.items():
        print("Processing RMND tech `{}`".format(tech))

        # region specific techs that are adressed
        actvts = [act for act in all_tech_actvts if act["location"] in regions]
        
        if len(actvts) == 0:
            print("No activities found for {}".format(tech))
            continue
        
        demand = 1.
        
        # CHP plants are a problem, not only because they provide energy in megajoule units
        # find out which part is electric energy
        if "CHP" in tech:
            demand = 1./3.6
            
        # normal multi-region and multi-tech lca
        lca = ei2rmnd.multi_lca_average(actvts, demand)
        
        # ... and remove the double-counting
        lca = ei2rmnd.remove_double_counting(lca, actvts, all_actvts)
        
        # bionames are only generated once. This takes some time.
        # probably these never change...
        if len(bio_names) != lca.inventory.shape[0]:
            print("Bionames changed, old length: {}".format(len(bio_names)))
            bio_names = [get_activity(key)["name"] for key in lca.biosphere_dict]
            print("new length: {}".format(len(bio_names)))

        # flows are aggregated by the resp. material
        # we do not care *where* the material flows to
        result[tech] = pd.DataFrame.from_dict({
            "flow": bio_names,
            "amount": [lca.inventory[n, :].sum() for n in range(len(bio_names))]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [154]:
df = rmnd_bioflows_without_double_counting("EUR")

N/A% (0 of 20) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']


100% (20 of 20) |########################| Elapsed Time: 0:00:57 Time:  0:00:57


Processing RMND tech `Biomass IGCC CCS`
Bionames changed, old length: 0
new length: 2079
Processing RMND tech `Biomass IGCC`
Processing RMND tech `Coal IGCC`
Processing RMND tech `Coal IGCC CCS`
Processing RMND tech `Coal PC CCS`
Processing RMND tech `Gas CCS`
Processing RMND tech `Biomass CHP`
Processing RMND tech `Coal PC`
Processing RMND tech `Coal CHP`
Processing RMND tech `Gas OC`
Processing RMND tech `Gas CC`
Processing RMND tech `Gas CHP`
Processing RMND tech `Geothermal`
Processing RMND tech `Hydro`
Processing RMND tech `Hydrogen`
No activities found for Hydrogen
Processing RMND tech `Nuclear`
Processing RMND tech `Oil`
Processing RMND tech `Solar CSP`
Processing RMND tech `Solar PV`
Processing RMND tech `Wind`


In [167]:
df = df.reset_index().rename({"level_0": "RMND Tech"}, axis=1).set_index(["RMND Tech", "flow"])
df.sample(20)

amount
RMND Tech     flow                                                            
Gas OC        Metolachlor                                         6.585789e-11
Geothermal    4-Methyl-2-pentanol                                 1.292289e-19
Coal CHP      Iprodion                                            2.057672e-14
              Copper, 1.18% in sulfide, Cu 0.39% and Mo 8.2E-...  4.914971e-07
Solar CSP     Transformation, from dump site, slag compartment    8.271584e-09
Coal IGCC     Silver, Ag 1.5E-4%, Au 6.8E-4%, in ore, in ground   3.145078e-14
Geothermal    4-Methyl-2-pentanone                                6.362139e-13
Gas CHP       Ruthenium-103                                       1.753225e-16
Coal IGCC CCS Phosphorus oxychloride                              0.000000e+00
Hydro         Methane, tetrafluoro-, R-14                         7.616569e-11
Biomass IGCC  Phenmedipham                                        8.784113e-15
Wind          Diethylamine                                        1.950322e-13
Biomass CHP   Fresh water (obsolete)                              1.027876e-09
Oil           Trifluralin                                         8.145729e-12
              Gold, Au 1.4E-4%, in ore, in ground                 1.445069e-11
Geothermal    Rhenium, in crude ore, in ground                    1.116567e-13
Gas OC        Carbonate                                           4.137311e-09
Coal IGCC CCS Dimethylamine                                       6.673708e-13
Biomass CHP   Transformation, to pasture, man made, extensive     4.413526e-15
Coal IGCC     Mefenpyr-diethyl                                    4.672284e-21

## Bioflows from the electricity sector with double counting

E.g., flows from wind power plants include coal-generated electricity impacts that are already being accounted for by a `electricity production, coal` lca.

In [105]:
def rmnd_bioflows_with_double_counting(region, scenario="BAU", year=2030, tech_primers={}):
    """ Extract bioflows for the REMIND electricity sector in a REMIND region.
        A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
        REMIND technology x.
    """
    eidb_name = ei2rmnd.get_REMIND_database_name(scenario, year)
    eidb = Database(eidb_name)

    result = {}
    bio_names = []
    regions = ei2rmnd.ei_locations_in_remind_region(region)
    print("Found following regions for {}: {}".format(region, regions))
    for key in eimod.available_electricity_generating_technologies:
        print("Processing RMND Technology {}".format(key))
        if key in tech_primers:
            techs = [tech_primers[key]]
        else:
            techs = eimod.available_electricity_generating_technologies[key]
        if len(techs) == 0:
            print("No technologies available for {}! Skipping.".format(key))
            continue
        units = {
            "kilowatt hour": 1.,
            "megajoule": 1./3.6
        }
        lca = ei2rmnd.lca_for_multiple_techs_and_regions(
                techs, regions, eidb, units_and_conversions=units)
        
        # bionames are only generated once. This takes some time.
        # probably these never change...
        if len(bio_names) != lca.inventory.shape[0]:
            print("Bionames changed, old length: {}".format(len(bio_names)))
            bio_names = [get_activity(key)["name"] for key in lca.biosphere_dict]
            print("new length: {}".format(len(bio_names)))

        # flows are aggregated by the resp. material
        # we do not care *where* the material flows to
        result[key] = pd.DataFrame.from_dict({
            "flow": bio_names,
            "amount": [lca.inventory[n, :].sum() for n in range(len(bio_names))]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [106]:
df_dc = rmnd_bioflows_with_double_counting("EUR")

Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']
Processing RMND Technology Biomass IGCC CCS
Bionames changed, old length: 0
new length: 2079
Processing RMND Technology Biomass IGCC
Processing RMND Technology Coal IGCC
Processing RMND Technology Coal IGCC CCS
Processing RMND Technology Coal PC CCS
Processing RMND Technology Gas CCS
Processing RMND Technology Biomass CHP
Processing RMND Technology Coal PC
Processing RMND Technology Coal CHP
Processing RMND Technology Gas OC
Processing RMND Technology Gas CC
Processing RMND Technology Gas CHP
Processing RMND Technology Geothermal
Processing RMND Technology Hydro
Processing RMND Technology Hydrogen
No technologies available for Hydrogen! Skipping.
Processing RMND Technology Nuclear
Processing RMND Technology Oil


In [110]:
df_dc.index = df_dc.index.rename(["RMND Tech", "flow"])

In [111]:
df_dc.sort_index()[:30]

amount
RMND Tech   flow                                               
Biomass CHP 1,3-Dioxolan-2-one                     5.504127e-10
            1,4-Butanediol                         6.419200e-12
            1-Pentanol                             6.018845e-13
            1-Pentene                              7.954006e-12
            2,2,4-Trimethyl pentane                4.339602e-15
            2,4-D                                  6.614032e-09
            2,4-D amines                           7.723629e-20
            2,4-D ester                            5.003715e-20
            2,4-DB                                 5.413703e-20
            2,4-di-tert-butylphenol                0.000000e+00
            2-Aminopropanol                        5.819360e-13
            2-Methyl pentane                       6.967736e-10
            2-Methyl-1-propanol                    1.777723e-12
            2-Methyl-2-butene                      3.223782e-16
            2-Nitrobenzoic acid                    3.030090e-13
            2-Propanol                             1.685840e-08
            2-chlorobenzaldehyde                   0.000000e+00
            3-Methyl-1-butanol                     0.000000e+00
            4-Methyl-2-pentanol                    1.236453e-19
            4-Methyl-2-pentanone                   1.708335e-12
            AOX, Adsorbable Organic Halogen as Cl  4.105488e-09
            Abamectin                              0.000000e+00
            Acenaphthene                           1.326423e-12
            Acenaphthylene                         1.224729e-13
            Acephate                               1.572855e-10
            Acetaldehyde                           3.569259e-07
            Acetamide                              2.311316e-11
            Acetamiprid                            0.000000e+00
            Acetic acid                            5.014071e-08
            Acetochlor                             1.259551e-13

## Impact of double counting

Compare versions with and without double counting.

In [168]:
fulldf = df.join(df_dc, lsuffix="_nodc", rsuffix="_dc")

In [169]:
fulldf["rel. (nodc/dc)"] = fulldf["amount_nodc"]/fulldf["amount_dc"]

In [170]:
fulldf["abs. (dc - nodc)"] = fulldf["amount_dc"] - fulldf["amount_nodc"]

In [171]:
fulldf.sort_values("abs. (dc - nodc)", ascending=False)[:30]

,,amount_nodc,amount_dc,rel. (nodc/dc),abs. (dc - nodc)
RMND Tech,flow,,,,
Biomass IGCC,Radon-222,7.459086e-04,18.179998,0.000041,18.179252
Biomass IGCC CCS,Radon-222,5.198503e-04,9.628468,0.000054,9.627948
Biomass IGCC,"Noble gases, radioactive, unspecified",2.243657e-05,3.947591,0.000006,3.947569
Geothermal,Radon-222,9.184669e-05,3.482510,0.000026,3.482418
Solar PV,Radon-222,5.831493e-04,3.478273,0.000168,3.477690
Biomass CHP,"Energy, gross calorific value, in biomass",1.329324e+00,4.631017,0.287048,3.301693
Coal PC CCS,Radon-222,7.723146e-03,2.533320,0.003049,2.525597
Coal IGCC CCS,Radon-222,3.840599e-03,2.288813,0.001678,2.284972
Biomass IGCC CCS,"Noble gases, radioactive, unspecified",1.321877e-05,2.105826,0.000006,2.105813


In [172]:
fulldf.loc[fulldf["rel. (nodc/dc)"] > 1].sort_values(by="rel. (nodc/dc)", ascending=False)

amount_nodc  \
RMND Tech        flow                                                               
Biomass IGCC     Transformation, to dump site, residual material... -2.933052e-08   
                 Transformation, from dump site, residual materi... -2.933344e-08   
Biomass IGCC CCS Cadmium                                            -2.554539e-09   
Coal IGCC        Transformation, to dump site, sanitary landfill    -2.287068e-09   
                 Transformation, from dump site, sanitary landfill  -2.287068e-09   
Coal IGCC CCS    Transformation, from dump site, inert material ... -2.307214e-08   
                 Transformation, to dump site, inert material la... -2.307214e-08   
Biomass IGCC CCS Lead                                               -1.641552e-07   
Coal IGCC CCS    Cyanide                                            -4.721098e-09   
Biomass IGCC CCS Calcium, ion                                       -1.684903e-03   
Gas CCS          Antimony                                            1.688398e-08   
Biomass IGCC CCS Aluminium                                          -1.351901e-04   
                 Beryllium                                          -1.323643e-07   
                 Manganese                                          -8.709922e-05   
                 Hydrogen sulfide                                   -4.067520e-06   
Coal IGCC CCS    Transformation, from pasture, man made             -3.634913e-06   
Coal IGCC        Cyanide                                            -5.156816e-09   
Biomass IGCC CCS Titanium, ion                                      -4.649831e-06   
Coal PC CCS      Transformation, to dump site, residual material... -2.911734e-08   
                 Transformation, from dump site, residual materi... -2.911847e-08   
Coal IGCC CCS    Tungsten                                           -3.097941e-08   
Coal IGCC        Transformation, from pasture, man made             -3.009112e-06   
Biomass IGCC     Carbon                                             -7.738301e-06   
                 Beryllium                                          -3.567833e-07   
Coal PC CCS      Iodide                                             -6.439882e-09   
Coal IGCC        Tungsten                                           -2.985480e-08   
Gas OC           Antimony                                            2.315695e-08   
Nuclear          Antimony                                            4.393911e-08   
Gas CC           Antimony                                            1.432330e-08   
Biomass IGCC     Oxygen                                             -7.641486e-04   
...                                                                           ...   
Oil              Glutaraldehyde                                      1.958288e-09   
                 Transformation, from seabed, unspecified            2.547325e-05   
                 Indeno(1,2,3-cd)pyrene                              1.173073e-15   
                 Transformation, to permanent crop, irrigated        2.687053e-08   
                 Transformation, from permanent crop, irrigated      2.687053e-08   
                 Toluene                                             3.158967e-06   
                 Hydrogen                                            3.651633e-07   
Coal IGCC        COD, Chemical Oxygen Demand                        -1.066807e-03   
Oil              Borate                                              1.079694e-10   
Coal IGCC CCS    Transformation, to dump site, residual material... -2.257272e-06   
                 Transformation, from dump site, residual materi... -2.257273e-06   
Oil              Clay, bentonite, in ground                          6.724758e-05   
                 Lead                                                8.745010e-07   
                 Carbon monoxide, fossil                             3.009074e-04   
                 Particulates, > 2.5 um, and < 10um                  9.9185